In [1]:
import pandas as pd

In [2]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models.tools import HoverTool
from bokeh.models import PrintfTickFormatter

In [3]:
plt.style.use("fivethirtyeight")   # ‘dark_background', ''bmh', 'grayscale', 'ggplot', 'fivethirtyeight'

In [4]:
def get_data():
    ''' read data from JHU CSSE website and return the pandas dataframe
    '''
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
    df = pd.read_csv(url)
    
    return df

In [5]:
df = get_data()
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,82,114,147,177,212,272,322,411,599,599.0
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,773,839,825,878,889,924,963,1007,1086,1086.0
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,212,226,243,266,313,345,385,432,455,455.0
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,2,2.0
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,238,428,566,673,790,900,1030,1183,1306,1306.0


In [6]:
ltime = pd.Timestamp(df.columns[-1])
print(f'The last date of update is {ltime:%b %d, %Y}')

The last date of update is Mar 23, 2020


In [7]:
def data_1cty(df, cty):
    ''' derive country sum daily cases
    
    Parameters
    ----------
    df : pd dataframe
        the input data
    cty : str
        the country name
        
    Returns
    -------
    series : pd Series
        the total daily cases for the given country
    '''
    
    filt = (df['Country/Region'] == cty)
    nsts = filt.sum()
    if nsts == 0:
        return None
    else:
        series = df.loc[filt].iloc[:,4:].sum(axis=0)
        series.index = pd.DatetimeIndex(series.index)
        return series

In [8]:
def data_countries(df):
    df_ctr = pd.DataFrame()    
    str_ctrs = set(list(df['Country/Region'].values))
    for ctr in str_ctrs:
        df_ctr[ctr] = data_1cty(df, ctr)
    return df_ctr

In [9]:
df_ctr = data_countries(df)

In [10]:
def plot_ctrs_ts(df_ctr, ctr_nm_all, colors=['blue', 'green', 'red', 'cyan', 'magenta', 'yellow'] ):
    output_notebook()
    source = ColumnDataSource(df_ctr)
    
    p = figure(plot_width=900, plot_height=500, x_axis_type='datetime',y_axis_type='log')
    p.title.text = 'CoV19 cases'
    p.xaxis.axis_label='Date'
    p.yaxis.axis_label='Number of cases'
    p.yaxis.formatter = PrintfTickFormatter(format="%5f")
    
    for ctr_nm, c in zip(ctr_nm_all,colors):
        p.line(x='index',y=ctr_nm,source=source, line_width=2, color=c, alpha=0.4)
        ttp = [("date", "$x{%F}"),("cases", "@"+ctr_nm+"{int}"),("country",ctr_nm)]        
        cr = p.circle(x='index',y=ctr_nm,source=source,fill_color=c,size=8,alpha=0.4,line_color=None,
                     hover_fill_color='firebrick',hover_line_color='firebrick',
                     legend_label=ctr_nm, muted_color=c, muted_alpha=1)
        fmt = {'$x': 'datetime'}
        p.add_tools(HoverTool(tooltips=ttp, renderers=[cr],formatters=fmt))
    
    p.legend.location='top_left'
    p.legend.click_policy='mute'
    show(p)

In [11]:
plot_ctrs_ts(df_ctr, ['US','Italy','Spain'])

Loading BokehJS ...